# ハイブリッド検索

LangChainでの標準的な検索は、ベクトル類似性に基づいて行われます。  
ただし、いくつかのベクトルストア（Astra DB、ElasticSearch、Neo4J、AzureSearch、Qdrantなど）では、ベクトル類似性検索と他の検索技術（全文検索、BM25など）を組み合わせた高度な検索をサポートしています。  
これを一般的に「ハイブリッド検索」と呼びます。

**Step 1: 使用するベクトルストアがハイブリッド検索をサポートしているか確認する**

現在、LangChainにはハイブリッド検索を統一的に実行する方法はありません。ベクトルストアごとに独自の方法が用意されている場合があります。  
これは通常、`similarity_search` 実行時にキーワード引数として指定されます。

ドキュメントやソースコードを読んで、使用するベクトルストアがハイブリッド検索をサポートしているか、またその使用方法を確認してください。

**Step 2: そのパラメータをチェーンの設定可能なフィールドとして追加する**

設定可能なフィールドとして追加することで、実行時にチェーンを簡単に呼び出し、関連するフラグを設定できます。
設定に関する詳細はこちらのドキュメントをご覧ください。 [this documentation](/docs/how_to/configure)

**Step 3: 設定可能なフィールドを使用してチェーンを呼び出す**

実行時に、このチェーンを設定可能なフィールドとともに呼び出します。

## コード例

具体的なコード例を見てみましょう。この例では、Astra DBのCassandra/CQLインターフェースを使用します。

以下のPythonパッケージをインストールしてください：

In [ ]:
!pip install "cassio>=0.1.7"

接続情報を取得 [connection secrets](https://docs.datastax.com/en/astra/astra-db-vector/get-started/quickstart.html).

Cassioを初期化:

In [ ]:
import cassio

cassio.init(
    database_id="Your database ID",
    token="Your application token",
    keyspace="Your key space",
)

標準インデックスアナライザー [index analyzer](https://docs.datastax.com/en/astra/astra-db-vector/cql/use-analyzers-with-cql.html)を使用してCassandra VectorStoreを作成します。  
このインデックスアナライザーは、単語のマッチングを有効にするために必要です。

In [ ]:
from cassio.table.cql import STANDARD_ANALYZER
from langchain_community.vectorstores import Cassandra
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = Cassandra(
    embedding=embeddings,  # OpenAI埋め込みモデルを使用
    table_name="test_hybrid",  # テーブル名
    body_index_options=[STANDARD_ANALYZER],  # 標準アナライザーを設定
    session=None,  # セッション情報
    keyspace=None,  # キースペース情報
)

# テキストをベクトルストアに追加
vectorstore.add_texts(
    [
        "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
    ]
)


以下のコードでは、標準的な類似検索を行います。すべてのドキュメントが返されます。

In [ ]:
vectorstore.as_retriever().invoke("What city did I visit last?")

[Document(page_content='In 2022, I visited New York'),
Document(page_content='In 2023, I visited Paris'),
Document(page_content='In 2021, I visited New Orleans')]

`body_search` 引数を使用して、検索を「new」という単語に基づいてフィルタリングします。

In [ ]:
vectorstore.as_retriever(search_kwargs={"body_search": "new"}).invoke(
    "What city did I visit last?"
)

[Document(page_content='In 2022, I visited New York'),
Document(page_content='In 2021, I visited New Orleans')]

質問応答用のチェーンを作成

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnablePassthrough,
)
from langchain_openai import ChatOpenAI

質問応答用の基本チェーン設定

In [ ]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retriever = vectorstore.as_retriever()

ここでは、リトリーバーに設定可能なフィールドを持たせています。  
すべてのベクトルストアリトリーバーには search_kwargs というフィールドがあります。  
これは、ベクトルストア固有のフィールドを含む単なる辞書です。

In [ ]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

設定可能なリトリーバーを使用してチェーンを作成できるようになりました。

In [ ]:
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("What city did I visit last?")

Paris

設定可能なオプションを使用してチェーンを呼び出すことができます。  
`search_kwargs` は設定可能なフィールドのIDです。この値には、Astra DBで使用する検索引数を指定します。

In [ ]:
chain.invoke(
    "What city did I visit last?",
    config={"configurable": {"search_kwargs": {"body_search": "new"}}},
)

New York